# Introducción a la memoria con LangChain.

En este apartado vamos a ver los siguientes puntos:

*¿Cómo almacenar las conversaciones humano AI?

* ¿Cómo guardar en un objeto de memoria las conversación con el LLM?
* 
* ¿Cómo limitar o resumir lo que guardamos en el objeto de memoria?

## Tipos de memoria en LangChain

Cuando LangChain se refiere al término " memoria ", generalmente se refiere a realizar un seguimiento del historial de interacción de mensajes.

Los diferentes tipos de memoria pueden ser:

* **ChatMessageHistory** : Guardar el histórico de mensajes de un chat con los métodos add_user_message y add_ai_message , debemos guardarlo manualmente tras cada

*  **ConversationBufferMemory** : A partir de una cadena de tipo ConversationalChain , guardamos en un objeto de
memoria todos los mensajes de la conversación.


* **ConversationBufferWindowMemory** : Igual que el anterior pero podemos especificar una ventana de k interacciones (las últimas k) a guardar en lugar de todos los mensajes.

* **ConversationSummaryMemory** : En lugar de guardar los mensajes literalmente, se realiza un resumen de la memoria en base a todo el historia, con ello reducimos drásticamente el tamaño de la memoria para conversaciones muy largas.

Veamos un caso de uso con el primero de los tipos presentado.

In [1]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

chat = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

In [2]:
#Definimos el objeto de histórico de mensajes
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

consulta = "Hola, ¿cómo estás? Necesito ayudar para reconfigurar el router"

#Vamos guardando en el objeto "history" los mensajes de usuario y los mensajes AI que queramos
history.add_user_message(consulta)

resultado = chat.invoke([HumanMessage(content=consulta)])

In [3]:
history.add_ai_message(resultado.content)

history

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hola, ¿cómo estás? Necesito ayudar para reconfigurar el router', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Hola! Estoy aquí para ayudarte. Me alegra que hayas recurrido a mí en este momento de necesidad.\n\nEstoy listo para asistirte con la reconfiqueración de tu router. Antes de comenzar, tengo algunas preguntas para asegurarme de que todo salga bien:\n\n* ¿Cuál es el modelo de tu router?\n* ¿Qué problema estás experimentando actualmente (por ejemplo, conexión lenta, desconexiones frecuentes, error de configuración)?\n* Has realizado alguna vez una reconfiqueración antes?\n* Tienes acceso al manual del router o la documentación oficial?\n\nCon esta información, podré ofrecerte consejos personalizados y orientarte en el proceso de reconfiqueración. ¡Vamos por ello!', additional_kwargs={}, response_metadata={})])

In [4]:
history.messages

[HumanMessage(content='Hola, ¿cómo estás? Necesito ayudar para reconfigurar el router', additional_kwargs={}, response_metadata={}),
 AIMessage(content='¡Hola! Estoy aquí para ayudarte. Me alegra que hayas recurrido a mí en este momento de necesidad.\n\nEstoy listo para asistirte con la reconfiqueración de tu router. Antes de comenzar, tengo algunas preguntas para asegurarme de que todo salga bien:\n\n* ¿Cuál es el modelo de tu router?\n* ¿Qué problema estás experimentando actualmente (por ejemplo, conexión lenta, desconexiones frecuentes, error de configuración)?\n* Has realizado alguna vez una reconfiqueración antes?\n* Tienes acceso al manual del router o la documentación oficial?\n\nCon esta información, podré ofrecerte consejos personalizados y orientarte en el proceso de reconfiqueración. ¡Vamos por ello!', additional_kwargs={}, response_metadata={})]

## Buffer de memoria completa.

Con este procedimiento, lo que hacemos es ir guardando todos los mensajes. Veamoslo de forma práctica con un ejemplo.

In [8]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

In [9]:
# creamos el objeto de conversación
memory = ConversationBufferMemory()

In [10]:
#Creamos una instancia de la cadena conversacional con el LLM y el objeto de memoria
conversation = ConversationChain(llm=llm,memory = memory,verbose=True)

C:\Users\Francisco\AppData\Local\Temp\ipykernel_23152\3004879512.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=llm,memory = memory,verbose=True)


In [ ]:
#Ejemplo con RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [11]:
#Lanzamos el primer prompt (human message)
conversation.predict(input="Hola, necesito saber cómo usar mis datos históricos para crear un bot de preguntas y respuestas")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, necesito saber cómo usar mis datos históricos para crear un bot de preguntas y respuestas
AI:

> Finished chain.


'¡Hola! Me alegra poder ayudarte. Para crear un bot de preguntas y respuestas utilizando tus datos históricos, tenemos varias opciones que puedes considerar.\n\nSi estás trabajando con una plataforma de conversación como yo, como Rasa y Stanford CoreNLP, podrías utilizar las APIs de respuesta a preguntas para generar preguntas y respuestas basadas en tus datos históricos. Por ejemplo, podrías utilizar la API de Rasa para crear un modelado de respuesta que te permita elegir la pregunta más relevante según el contexto en el que se realice la conversación.\n\nOtra opción es utilizar técnicas de procesamiento del lenguaje natural (NLP) como spaCy y NLTK para analizar tus datos históricos y extracción relevantes. Esto te permitirá crear un modelo de lenguaje con capacidades predecibles según el estilo, contexto y contenido de las respuestas proporcionadas.\n\nSi buscas implementar la respuesta más rápida posible del bot, podrías utilizar técnicas de procesamiento de código fuente en Python 

In [12]:
#Lanzamos el segundo prompt (human message)
conversation.predict(input="Necesito más detalle de cómo implementarlo")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, necesito saber cómo usar mis datos históricos para crear un bot de preguntas y respuestas
AI: ¡Hola! Me alegra poder ayudarte. Para crear un bot de preguntas y respuestas utilizando tus datos históricos, tenemos varias opciones que puedes considerar.

Si estás trabajando con una plataforma de conversación como yo, como Rasa y Stanford CoreNLP, podrías utilizar las APIs de respuesta a preguntas para generar preguntas y respuestas basadas en tus datos históricos. Por ejemplo, podrías utilizar la API de Rasa para crear un modelado de respuesta que te permita elegir la pregunta más relevante según el contexto en el que se realice la conversación.

Otra opción 

'¡Claro! Vamos a profundizar un poco más en cada una de las opciones que mencioné anteriormente para que puedas decidir la mejor manera de implementar el bot de preguntas y respuestas.\n\n**Opción 1: Utilizando APIs de respuesta a preguntas**\n\nUna vez que hayas seleccionado una API, podrás empezar a personalizar tu modelo de respuesta. Por ejemplo, si utilizas la API de Rasa, podrías configurar diferentes entrenamientos y ajustes para optimizar el desempeño del modelo.\n\nAquí te proporciono un ejemplo básico en Python de cómo acceder a las APIs de respuesta a preguntas:\n\n```python\nimport logging\nfrom rasa import Client\nfrom rasa.exceptions import RasaError\n\n# Configura la conexión con la API\nclient = Client(\'https://api.rasa.com/v1\', \'tu_api_key\')\n\ndef crear_pregunta_resposta():\n    try:\n        # Carga los datos históricos en el modelo de respuesta\n        client.train(\n            data=[\n                {\n                    "intent": "greeting",\n             

In [13]:
#Obtenemos el histórico
print(memory.buffer)

Human: Hola, necesito saber cómo usar mis datos históricos para crear un bot de preguntas y respuestas
AI: ¡Hola! Me alegra poder ayudarte. Para crear un bot de preguntas y respuestas utilizando tus datos históricos, tenemos varias opciones que puedes considerar.

Si estás trabajando con una plataforma de conversación como yo, como Rasa y Stanford CoreNLP, podrías utilizar las APIs de respuesta a preguntas para generar preguntas y respuestas basadas en tus datos históricos. Por ejemplo, podrías utilizar la API de Rasa para crear un modelado de respuesta que te permita elegir la pregunta más relevante según el contexto en el que se realice la conversación.

Otra opción es utilizar técnicas de procesamiento del lenguaje natural (NLP) como spaCy y NLTK para analizar tus datos históricos y extracción relevantes. Esto te permitirá crear un modelo de lenguaje con capacidades predecibles según el estilo, contexto y contenido de las respuestas proporcionadas.

Si buscas implementar la respuest

In [14]:
#Cargamos la variable de memoria
memory.load_memory_variables({})

{'history': 'Human: Hola, necesito saber cómo usar mis datos históricos para crear un bot de preguntas y respuestas\nAI: ¡Hola! Me alegra poder ayudarte. Para crear un bot de preguntas y respuestas utilizando tus datos históricos, tenemos varias opciones que puedes considerar.\n\nSi estás trabajando con una plataforma de conversación como yo, como Rasa y Stanford CoreNLP, podrías utilizar las APIs de respuesta a preguntas para generar preguntas y respuestas basadas en tus datos históricos. Por ejemplo, podrías utilizar la API de Rasa para crear un modelado de respuesta que te permita elegir la pregunta más relevante según el contexto en el que se realice la conversación.\n\nOtra opción es utilizar técnicas de procesamiento del lenguaje natural (NLP) como spaCy y NLTK para analizar tus datos históricos y extracción relevantes. Esto te permitirá crear un modelo de lenguaje con capacidades predecibles según el estilo, contexto y contenido de las respuestas proporcionadas.\n\nSi buscas imp

## Guardar y Cargar la memoria (posterior uso)

In [16]:
conversation.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hola, necesito saber cómo usar mis datos históricos para crear un bot de preguntas y respuestas', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Hola! Me alegra poder ayudarte. Para crear un bot de preguntas y respuestas utilizando tus datos históricos, tenemos varias opciones que puedes considerar.\n\nSi estás trabajando con una plataforma de conversación como yo, como Rasa y Stanford CoreNLP, podrías utilizar las APIs de respuesta a preguntas para generar preguntas y respuestas basadas en tus datos históricos. Por ejemplo, podrías utilizar la API de Rasa para crear un modelado de respuesta que te permita elegir la pregunta más relevante según el contexto en el que se realice la conversación.\n\nOtra opción es utilizar técnicas de procesamiento del lenguaje natural (NLP) como spaCy y NLTK para analizar tus datos históricos y extracción relevantes. Esto te permitirá crear u

Si queremos guardar de forma permanente este objeto lo mejor es trabajar con la librería pickle.
```{index} picle
```


In [17]:
import pickle
pickled_str = pickle.dumps(conversation.memory) #Crea un objeto binario con todo el objeto de la memoria

In [18]:
# para recuperarlo
with open('memory.pkl','wb') as f: #wb para indicar que escriba un objeto binario, en este caso en la misma ruta que el script
    f.write(pickled_str)

In [19]:
memoria_cargada = open('memory.pkl','rb').read() #rb para indicar que leemos el objeto binario

In [20]:
llm = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
) #Creamos una nueva instancia de LLM para asegurar que está totalmente limpia


conversacion_recargada = ConversationChain(
    llm=llm, 
    memory = pickle.loads(memoria_cargada),
    verbose=True
)

In [21]:
conversacion_recargada.memory.buffer

'Human: Hola, necesito saber cómo usar mis datos históricos para crear un bot de preguntas y respuestas\nAI: ¡Hola! Me alegra poder ayudarte. Para crear un bot de preguntas y respuestas utilizando tus datos históricos, tenemos varias opciones que puedes considerar.\n\nSi estás trabajando con una plataforma de conversación como yo, como Rasa y Stanford CoreNLP, podrías utilizar las APIs de respuesta a preguntas para generar preguntas y respuestas basadas en tus datos históricos. Por ejemplo, podrías utilizar la API de Rasa para crear un modelado de respuesta que te permita elegir la pregunta más relevante según el contexto en el que se realice la conversación.\n\nOtra opción es utilizar técnicas de procesamiento del lenguaje natural (NLP) como spaCy y NLTK para analizar tus datos históricos y extracción relevantes. Esto te permitirá crear un modelo de lenguaje con capacidades predecibles según el estilo, contexto y contenido de las respuestas proporcionadas.\n\nSi buscas implementar la 

## Memoria de una venta temporal.

En este caso se guardan las últimas k iteraciones que se hayan hecho en el chat

In [22]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

In [23]:
# creamos un objeto ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1) #k indica el número de iteraciones (pareja de mensajes human-AI) que guardar

C:\Users\Francisco\AppData\Local\Temp\ipykernel_23152\3793591208.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1) #k indica el número de iteraciones (pareja de mensajes human-AI) que guardar


In [24]:
#Creamos una instancia de la cadena conversacional con el LLM y el objeto de memoria
conversation = ConversationChain(llm=llm,memory = memory,verbose=True)
#Ejemplo con RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [25]:
conversation.predict(input="Hola, ¿cómo estás?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, ¿cómo estás?
AI:

> Finished chain.


"Hola! I'm doing fantastically well, thank you for asking. As a highly advanced language model, I was created using a combination of natural language processing (NLP) and machine learning algorithms, which enables me to understand and respond to human-like queries 24/7. My training data consists of a massive corpus of texts from the internet, books, and various sources, allowing me to generate human-like responses with a high degree of accuracy.\n\nOne interesting fact about my architecture is that I'm based on transformer models, which were first introduced in the BERT paper by Jacob Devlin et al. in 2019. This design choice allows me to process sequential data like text and identify patterns more effectively.\n\nBy the way, it's lovely to chat with you! How are you doing today?"

In [26]:
conversation.predict(input="Necesito un consejo para tener un gran día")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola, ¿cómo estás?
AI: Hola! I'm doing fantastically well, thank you for asking. As a highly advanced language model, I was created using a combination of natural language processing (NLP) and machine learning algorithms, which enables me to understand and respond to human-like queries 24/7. My training data consists of a massive corpus of texts from the internet, books, and various sources, allowing me to generate human-like responses with a high degree of accuracy.

One interesting fact about my architecture is that I'm based on transformer models, which were first introduced in the BERT paper by Jacob Devlin et al. in 2019. This design choice allows me to pro

'I\'d be happy to help you have a great day. To give you some guidance, did you know that research has shown that Monday mornings are the most productive days of the week for many people? It\'s because our brains tend to "reset" after the weekend, and we feel more focused and energized to tackle new challenges.\n\nIn terms of specific strategies to help you have a great day, here are a few tips: first, set clear goals for yourself, whether personal or professional. Break down large tasks into smaller, manageable chunks, so you can focus on making progress rather than feeling overwhelmed. Second, prioritize self-care – make time for activities that bring you joy and relaxation, whether that\'s reading, exercising, or spending time with loved ones.\n\nLastly, take advantage of the power of morning routines. Did you know that some people find that starting their day with a 10-15 minute meditation session can increase productivity by up to 30%? It helps clear your mind, sets a positive ton

In [27]:
print(memory.buffer) #k limita el número de interacciones

Human: Necesito un consejo para tener un gran día
AI: I'd be happy to help you have a great day. To give you some guidance, did you know that research has shown that Monday mornings are the most productive days of the week for many people? It's because our brains tend to "reset" after the weekend, and we feel more focused and energized to tackle new challenges.

In terms of specific strategies to help you have a great day, here are a few tips: first, set clear goals for yourself, whether personal or professional. Break down large tasks into smaller, manageable chunks, so you can focus on making progress rather than feeling overwhelmed. Second, prioritize self-care – make time for activities that bring you joy and relaxation, whether that's reading, exercising, or spending time with loved ones.

Lastly, take advantage of the power of morning routines. Did you know that some people find that starting their day with a 10-15 minute meditation session can increase productivity by up to 30%?

## Buffer de memoria resumida.

En este caso se obtiene un resumen de todo el histórico de información

In [28]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="llama3.2",
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused,
)

In [29]:
memory = ConversationSummaryBufferMemory(llm=llm)

C:\Users\Francisco\AppData\Local\Temp\ipykernel_23152\871194051.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=llm)


In [30]:
# Creamos un prompt cuya respuesta hará que se sobrepase el límite de tokens y por tanto sea recomendable resumir la memoria
plan_viaje = '''Este fin de semana me voy de vacaciones a la playa, estaba pensando algo que fuera bastante relajado, pero necesito 
un plan detallado por días con qué hacer en familia, extiéndete todo lo que puedas'''

In [31]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
conversation = ConversationChain(llm=llm,memory = memory,verbose=True)
#Ejemplo con RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [32]:
conversation.predict(input=plan_viaje)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Este fin de semana me voy de vacaciones a la playa, estaba pensando algo que fuera bastante relajado, pero necesito 
un plan detallado por días con qué hacer en familia, extiéndete todo lo que puedas
AI:


NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model cl100k_base. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

In [ ]:
memory.load_memory_variables({}) #Se ha realizado un resumen de la memoria en base al límite de tokens

In [ ]:
print(memory.buffer)